## Overview

This script takes as input the in-MEG behavioral data (which is trial based) and transforms it as events for MNE processing.

Each trial has several distinct events: 
 - trial onset
 - precue
 - each RSVP image (including the target on 'present' trials)
 - postcue
 - response
 - etc.
 
Each of these has a corresponding MEG code:

| Event                   | Code Value |
|-------------------------|------------|
| precue - cue shown      | 81         |
| precue - cue not shown  | 80         |
| object - flower         | 11         |
| object - car            | 12         |
| object - shoe           | 13         |
| object - chair          | 14         |
| scene - woods           | 21         |
| scene - bathroom        | 22         |
| scene - desert          | 23         |
| scene - coast           | 24         |
| target image            | ##9        |
| postcue - cue shown     | 181        |
| postcue - cue not shown | 180        |
| response screen         | 77         |
 
This structure makes it easy to categorize events by the type of condition they were a part of. Below is an example of a precued, target-present trial, for which the target was a flower, and that the subjct got correct:
 
 
| subject | trial_num | response_correct | cue_type | precue_type | postcue_type | target_identity | target_category | variable       | value |
|---------|-----------|------------------|----------|:-----------:|--------------|-----------------|-----------------|----------------|-------|
| s002    | 1         | 1                | precue   | precue      | nocue        | flower          | object          | precue         | 81    |
| s002    | 1         | 1                | precue   | precue      | nocue        | flower          | object          | picture1_value | 24    |
| s002    | 1         | 1                | precue   | precue      | nocue        | flower          | object          | picture2_value | 119   |
| s002    | 1         | 1                | precue   | precue      | nocue        | flower          | object          | picture3_value | 14    |
| s002    | 1         | 1                | precue   | precue      | nocue        | flower          | object          | picture4_value | 12    |
| s002    | 1         | 1                | precue   | precue      | nocue        | flower          | object          | picture5_value | 22    |
| s002    | 1         | 1                | precue   | precue      | nocue        | flower          | object          | picture6_value | 21    |
| s002    | 1         | 1                | precue   | precue      | nocue        | flower          | object          | postcue        | 180   |

It may seem redundant, but I can easily subset the MEG data into epochs for "picture 5" for which the target category was an object. 

All of the event/trial categorization should take place in this script. For example, if I wanted to look at the first half of trials, I'd add a column to the resulting events file for "exp_half [1/2]".

In [31]:
import os
import pandas as pd

In [32]:
data_path = '../../data/raw/'
infiles = [item for item in os.listdir(data_path) if item.endswith('txt')]

In [33]:
columns = ['trial_num','cue_type','target_identity','target_category','choices','target_presence',
           'response','response_correct','response_time','total_trial_time',
           'precue_value','IDUNNOpre1','IDUNNOpre2','precue_time','precue_time_actual','precue_position',
           'picture1_value','IDUNNO1','picture1_stim','picture1_time','picture1_time_actual', 'picture1_posititon',
           'picture2_value','IDUNNO2','picture2_stim','picture2_time','picture2_time_actual', 'picture2_posititon',
           'picture3_value','IDUNNO3','picture3_stim','picture3_time','picture3_time_actual', 'picture3_posititon',
           'picture4_value','IDUNNO4','picture4_stim','picture4_time','picture4_time_actual', 'picture4_posititon',
           'picture5_value','IDUNNO5','picture5_stim','picture5_time','picture5_time_actual', 'picture5_posititon',
           'picture6_value','IDUNNO6','picture6_stim','picture6_time','picture6_time_actual', 'picture6_posititon',
            'postcue_value','IDUNNOpost1','IDUNNOpost2','postcue_time','postcue_time_actual','postcue_position']

df_all = []
for infile in infiles:
    subject =  infile[:4]
    if subject not in  ['s002','s008']: #s002 is ok, just not the same format
        df = pd.read_csv(data_path+infile, sep="\t", header = None)
        df.columns = columns
        df['subject'] = subject
        df_all.append(df)
    elif subject == 's002':
        df2 = pd.read_csv(data_path+infile, sep="\t")
        df2['subject'] = subject
        
df2.columns = columns+['subject']
df_all.append(df2)

In [35]:
result = pd.concat(df_all)
result = result.reset_index(drop=True)

In [ ]:
Take the RSVP images and 

In [37]:
stimlist = result[['subject','picture1_value','picture2_value','picture3_value','picture4_value','picture5_value','picture6_value']]

In [38]:
stimlist = stimlist.apply(lambda x: x.astype(str).str.replace('[',''))
stimlist = stimlist.apply(lambda x: x.astype(str).str.replace(']',''))
stimlist = stimlist.apply(lambda x: x.astype(str).str.replace("'",''))

In [39]:
scene_dict = {'woods':1,'bathroom':2,'desert':3,'coast':4}
object_dict = {'flower':1,'car':2,'shoe':3,'chair':4}

In [40]:
stimlist.replace(r'\bwoods_\d*\b', 'woods', regex=True,inplace=True)
stimlist.replace(r'\bbathroom_\d*\b', 'bathroom', regex=True,inplace=True)
stimlist.replace(r'\bdesert_\d*\b', 'desert', regex=True,inplace=True)
stimlist.replace(r'\bcoast_\d*\b', 'coast', regex=True,inplace=True)

stimlist.replace(r'\bflower_\d*\b', 'flower', regex=True,inplace=True)
stimlist.replace(r'\bcar_\d*\b', 'car', regex=True,inplace=True)
stimlist.replace(r'\bshoe_\d*\b', 'shoe', regex=True,inplace=True)
stimlist.replace(r'\bchair_\d*\b', 'chair', regex=True,inplace=True)

In [41]:
stimlist['target_identity'] = result['target_identity']

In [42]:
stimlist.replace({'woods':21,'bathroom':22,'desert':23,'coast':24},inplace=True)
stimlist.replace({'flower':11,'car':12,'shoe':13,'chair':14},inplace=True)

In [43]:
stimlist = stimlist.reset_index(drop=True)

In [44]:
stimlist1 = pd.DataFrame(stimlist)
for i,row in stimlist1.iterrows():
    for item,key in zip(row.values[:6],stimlist1.columns[:6]):
        if item == row.values[7]:
            stimlist1.loc[i,key] = int(str(item)+'9')

In [45]:
stimlist1['cue_type'] = result['cue_type']
stimlist1['trial_num'] = result['trial_num']
stimlist1['response_correct'] = result['response_correct']
stimlist1['target_identity'] = result['target_identity']
stimlist1['target_category'] = result['target_category']

In [46]:
stimlist1['precue'] = 80
stimlist1['postcue'] = 180

stimlist1.loc[stimlist1.cue_type == 'precue','precue'] = 81
stimlist1.loc[stimlist1.cue_type == 'postcue','postcue'] = 181
stimlist1.loc[stimlist1.cue_type == 'doublecue','precue'] = 81
stimlist1.loc[stimlist1.cue_type == 'doublecue','postcue'] = 181

In [47]:
stimList_final = stimlist1[['subject','trial_num','response_correct','cue_type','target_identity','target_category',
                            'precue','picture1_value','picture2_value','picture3_value','picture4_value',
                            'picture5_value','picture6_value','postcue']]

In [48]:
stimList_final.head()

,subject,trial_num,response_correct,cue_type,target_identity,target_category,precue,picture1_value,picture2_value,picture3_value,picture4_value,picture5_value,picture6_value,postcue
0,s003,1,0,precue,chair,object,81,23,22,13,11,24,12,180
1,s003,2,0,nocue,False,False,80,23,13,12,24,21,11,180
2,s003,3,0,postcue,car,object,80,13,11,22,14,21,23,181
3,s003,4,0,postcue,desert,scene,80,14,22,11,12,21,24,181
4,s003,5,1,precue,desert,scene,81,14,24,13,21,12,22,180


In [49]:
stimList_final.loc[stimList_final.cue_type == 'precue','precue_type'] = 'precue'
stimList_final.loc[stimList_final.cue_type == 'postcue','precue_type'] = 'nocue'
stimList_final.loc[stimList_final.cue_type == 'nocue','precue_type'] = 'nocue'
stimList_final.loc[stimList_final.cue_type == 'doublecue','precue_type'] = 'precue'
stimList_final.loc[stimList_final.cue_type == 'precue','postcue_type'] = 'nocue'
stimList_final.loc[stimList_final.cue_type == 'postcue','postcue_type'] = 'postcue'
stimList_final.loc[stimList_final.cue_type == 'nocue','postcue_type'] = 'nocue'
stimList_final.loc[stimList_final.cue_type == 'doublecue','postcue_type'] = 'postcue'

In [50]:
df_list = pd.melt(stimList_final,
                  id_vars=['subject','trial_num','response_correct','cue_type',
                           'precue_type','postcue_type','target_identity',
                           'target_category'], 
                  value_vars=['precue','picture1_value','picture2_value','picture3_value',
                              'picture4_value','picture5_value','picture6_value',
                              'postcue'])

In [51]:
df_list = df_list.sort(['subject','trial_num']).reset_index(drop=True)

/Users/Amyunimus/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


In [53]:
df_list.head(10)

,subject,trial_num,response_correct,cue_type,precue_type,postcue_type,target_identity,target_category,variable,value
0,s002,31,1,precue,precue,nocue,flower,object,precue,81
1,s002,31,1,precue,precue,nocue,flower,object,picture1_value,24
2,s002,31,1,precue,precue,nocue,flower,object,picture2_value,119
3,s002,31,1,precue,precue,nocue,flower,object,picture3_value,14
4,s002,31,1,precue,precue,nocue,flower,object,picture4_value,12
5,s002,31,1,precue,precue,nocue,flower,object,picture5_value,22
6,s002,31,1,precue,precue,nocue,flower,object,picture6_value,21
7,s002,31,1,precue,precue,nocue,flower,object,postcue,180
8,s002,32,0,nocue,nocue,nocue,FALSE,FALSE,precue,80
9,s002,32,0,nocue,nocue,nocue,FALSE,FALSE,picture1_value,23


In [30]:
data_path_events = '../../data/processed/events/'

for subject in df_list.subject.unique():
    df_list[df_list.subject == subject][['response_correct','cue_type','precue_type','postcue_type',
                                         'target_identity','target_category','value']].to_csv(data_path_events+subject+'_events.txt',
                                                                            header=None, index=None)